In [ ]:
#Hello Danika, I created this extra folder for all the paper regarding tests. 
#I also copied the relevant drivers and wrappers into this folder since I did not want to mess with the original files in case I need to change something.
#I also changed the code in the way that it creates a file with all the raw data and adds the data immediately to the file.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
import csv
from Dispensing_wrapper_robot import *
from scale_driver import *
from motor_driver import *


SDK_VERSION: 1.13.19


In [11]:
coms = {
    'scaleCom': 'COM7',
    'motorsCom': 'COM9',
}

#Robot Connection
robot = Robot()
robot.initialize()
robot.GoTo_InitialPoint()

ROBOT_IP: 192.168.1.200, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 6,6,XI1303,MC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3
[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move


In [22]:
robot.GoTo_InitialPoint()

[SDK][ERROR][2024-06-03 14:50:54][base.py:374] - - API -> set_position -> code=9, pos=[-228.0, 0.0, 133.0, 0.0, 1.5707963267948966, 3.141592653589793], radius=-1, velo=20.0, acc=2000


In [7]:
#Motor Connection
motors = SerialConnection(coms['motorsCom'], 9600, 10)
dispenser_motor = Motor(motors, 1)

#Scale Connection
scale = Scale(coms['scaleCom'], 9600, 10)
#scale.connect() -> this will happen directly in the calibartion loop 
#scale.tare()

In [29]:
#This is for calibrating the scale. For it to work you must run the scale.connect function.
scale.connect()
scale.IntCal() 
scale.close()
#I will need to write in some wait function for the calibration to finish

In [19]:
scale.connect()
scale.tare()

Measurement(mode='N', value=0.0, unit='g', stable=True, message=None)
0.0


In [20]:
scale.close()

In [5]:
# Define your parameters
acceleration = 10
speed = 5
material = "HardCarbon"
version_inside = "Design2"
version_outside = "Design1"

In [8]:
dispenser_motor.move(600, True) 
#I ran some random tests and wrote some numbers down to get a feeling -> not looking good to be honest, i think with Super P we need a lot of turns to get a reproducibele outcome
#0.0063g per 200 steps
#0,0086 per 400
#0,0026 per 200
#0,0083 per 200
#0,0167 per 1000
#0,0172 per 1000
#0,0132 per 1000


In [12]:
#Error test: Test with Super P and Design 4, see at which point the error does not change anymore significantly 
#Motor is doing 200steps for one full turn, Design 4 has 12 pits which means 200/12 = 16.6 steps are neccesary to empty one pit
#Calibration step: 200/n, 2x200/n, 3x200/n, 4x200/n,...till teh first hole is being emptied (e.g 12 holes for design 4 so we empty six holes so the first hole which was filled gets emptied again) + 200, 1000,...

calibration = Calibration(acceleration, speed, material, version_inside, version_outside, dispenser_motor, scale, robot)

#calibration_steps = [17, 34, 51, 68, 85, 102, 200, 400, 600, 800, 1000, 2000] 200/12 = 16.6, so 17 steps, we could do factors of 17 like shown here
#calibration_steps = [17, 33, 50, 67, 83, 100, 200, 1000, 2000, 3000, 4000] # or we take factors of 16.6 and round up or down to the nearest integer liek shown here
#calibration_repeat = 10

#Design 2: 25pits 200/25
calibration_steps = [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 200, 400, 600]
calibration_repeat = 10

calibration.calibrate(calibration_steps, calibration_repeat, "Vial1")

calibration.save_calibration()

Repeat: 1
Scale opened


KeyboardInterrupt: 

In [9]:
#calibration_steps = [17, 33, 50, 67, 83, 100, 200, 1000, 2000, 3000, 4000]
#calibration_repeat = 10

calibration_steps = [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 200, 400, 600]
calibration_repeat = 10

material_per_step = 0.00005
total_steps = sum(calibration_steps)*calibration_repeat
total_material= sum(step*material_per_step for step in calibration_steps)*calibration_repeat
print(f'Total material used for calibration: {total_material}g')
print(f'Total number of steps: {total_steps}')

Total material used for calibration: 0.912g
Total number of steps: 18240
